In [1]:
! pip install peft bitsandbytes accelerate datasets

In [3]:
import os
from typing import List, Dict, Any

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, AutoConfig, set_seed
from torch.utils.data import Dataset
from datasets import load_dataset
from peft import PeftModel, get_peft_model, LoraConfig, prepare_model_for_kbit_training

set_seed(12, True)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

# Gradient Accumulation - 5 баллов

Давайте реализуем собственную аккумуляцию градиентов.
Ниже описано обучение обычного линейного слоя. Клеткой ниже этот код скопирован, там необходимо написать аккумуляцию ргадиентов.

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
input_size = 512
output_size = 256
batch_size = 64
gradient_accumulation_steps = 4



model = nn.Linear(input_size, output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

x = torch.randn(batch_size, input_size).to(device)
y = torch.randn(batch_size, output_size).to(device)
loss_fn = nn.MSELoss()
for i in range(1000):
    optimizer.zero_grad()
    output = model(x)
    loss = loss_fn(output, y)
    loss.backward()
    optimizer.step()

print(loss.item())

1.1878371238708496


Число шагов в аккумуляции определяется параметром gradient_accumulation_steps - это число шагов, которое мы хотим сделать перед оптимизацией.
Вам нужно поправить цикл обучения следующим образом:
1. Разбить текущий батч на gradient_accumulation_steps частей
2. Пройтись по каждому подбатчу (микробатчу), посчитать на нем функцию потерь, посчитать градиенты. Подумайте, нужно ли на что-либо делить или умножать функцию потерь, чтобы сохранился тот же масштаб обучения?
3. После прохождения всех микробатчей нужно сделать шаг оптимизации

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
input_size = 512
output_size = 256
batch_size = 64
gradient_accumulation_steps = 4



model = nn.Linear(input_size, output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

x = torch.randn(batch_size, input_size).to(device)
y = torch.randn(batch_size, output_size).to(device)
loss_fn = nn.MSELoss()
for i in range(1000):
    optimizer.zero_grad()
    chunks = x.chunk(gradient_accumulation_steps)
    chunk_loss = 0
    for chunk in chunks:
        output = model(x)
        chunk_loss += loss_fn(output, y)
    loss = chunk_loss / gradient_accumulation_steps
    loss.backward()
    optimizer.step()

print(loss.item())

1.1858601570129395


# QLORA - 15 баллов
Необходимо использовать аккумуляцию градиентов, чекпоинтинг активаций и обучение qlora.

In [7]:
model_name = "NousResearch/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
imdb = load_dataset("imdb")

Наша задача научиться генерировать класс текста posive или negative, чтобы сэкономить на fewshot промпте.

Давайте напишем collate_fn, которая собирает сэмпл следующим образом:

если текст имеет метку 1
`{text} ||| posive eos`
или
`{text} ||| negatve eos`
если текст имеет метку 0. (в качестве eos можно использовать tokenizer.eos_token_id)

Символы ||| нужны нам, чтобы разделить входной текст и метку, иначе модель может не понять, что нужно генерировать метку и продолжит генерировать текст. Таким образом мы научим модель после ||| генерировать положительный или отрицательнй отзыв стоит до этого.


Возвращать нужно словарь из 3х элементов:
1. input_ids - LongTensor токенов. В качестве паддинга нужно использовать tokenizer.eos_token_id.
2. attention_mask - LongTensor той же размерности, что и input_ids. 0 там, где стоят паддинги, 1 в остальных позициях
3. labels - метки, которые мы предсказыаем. Должен быть равен -100 на всех позициях, кроме позиций, которые соответствуют метке и eos символу.
Например
```python
tokenizer.encode("some text ||| positive </s>") # [1, 777, 1426, 3830, 29989, 6374, 2]
labels = [-100, -100, -100, -100, -100, 6374, 2]
```

Т.е. метки должны быть -100, кроме позиций, соответствующих предсказываемым токенам.

In [9]:
def collate_fn(batch: List[Dict[str, Any]]):
  """Function collates samples for training.

  сэмпл следующим образом
  если текст имеет метку 1
  {text} ||| posive eos
  или если текст имеет метку 0
  {text} ||| negatve eos
  в качестве eos можно использовать tokenizer.eos_token_id)"""
  class_mapping = {0: "negative", 1: "positive"}
  texts = [sample["text"] + " ||| " + class_mapping[sample["label"]] + " </s>" for sample in batch]

  input_ids = [tokenizer.encode(el) for el in texts]
  max_len = max(len(el) for el in input_ids)
  attention_mask = [[0] * (max_len - len(el)) + [1] * len(el) for el in input_ids]
  padded_input_ids = [[tokenizer.eos_token_id] * (max_len - len(el)) + el for el in input_ids]
  labels = [[-100] * (max_len-2) + el[-2:] for el in padded_input_ids]
  return {
      "input_ids": torch.tensor(padded_input_ids).long(),
      "attention_mask": torch.tensor(attention_mask).long(),
      "labels": torch.tensor(labels).long()
  }

res = collate_fn([imdb["train"][0], imdb["train"][12505], imdb["train"][2]])

assert tokenizer.decode(res["input_ids"][res["labels"] != -100]) == "negative</s> positive</s> negative</s>"

Далее нам нужно создать модель в nf4, т.е. 4-битной квантизации. Конфиг уже написан, нужно лишь подать его в модель. После этого нужно:
1. Создать конфиг адаптера LoraConfig (используйте r=8 или r=4, если будет OOM) и создать модель
2. Создать модель с адаптером с помощью PeftModel и LoraConfig
3. Чтобы обучение шло только по lora частям, нужно пройтись по всем параметрам модели с помощью model.named_parameters() и проставить у параметров, соответствующих lora атрибут requires_grad = True, а у всех остальных False

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
)

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
for name, module in model.named_modules():
  print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp
model.layers.2.mlp.gate_proj
model.l

In [12]:
torch.cuda.empty_cache()

In [19]:
peft_config = LoraConfig(
    r=8,
    # target_modules='all-linear',
    target_modules=["q_proj", "v_proj", "k_proj"],
    lora_dropout=.05
)
model = get_peft_model(model=model, peft_config=peft_config)
# не забудьте пройтись по всем параметрам и проставить .requires_grad там, где нужно

In [20]:
model.print_trainable_parameters()

trainable params: 6,291,456 || all params: 6,744,707,072 || trainable%: 0.0933


Осталось самое важное, аргументы обучения. Обязательно заполните следующие параметры:

1. Батч сайз и число шагов аккумуляции выставьте так, чтобы эффективный батч сайз был 16
2. Включите чекпоинтинг активаций

In [ ]:
args = TrainingArguments(
    gradient_checkpointing=True,
    # fsdp_config = {"activation_checkpointing": True}
    gradient_accumulation_steps=4,
    per_device_train_batch_size=4,
    report_to=None,
    remove_unused_columns=False,
    label_names=["labels"],
    max_steps=50
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=imdb["train"],
    tokenizer=tokenizer,
    data_collator=collate_fn,
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-22-1901ebbfb2a4>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Давайте протестируем, что модель что-то выучила

In [ ]:
input_text = imdb["test"][0]["text"] + " |||"
label = imdb["test"][0]["label"]
x = tokenizer(input_text, return_tensors="pt")
for k, v in x.items():
    x[k] = v.cuda()

print(label)
g = model.generate(**x, max_new_tokens=2, do_sample=False)
print(tokenizer.decode(g[0].tolist()))